In [2]:
import pandas as pd;
from sklearn.preprocessing import LabelEncoder
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CSE303/Lab-5/melbourne_housing_raw.csv')
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
for column in data.keys():
  if data[column].dtype == 'object':
    data[column] = label_encoder.fit_transform(data[column])
x=data.drop('Propertycount',axis=1)
y=data['Propertycount']
print(x.shape)
print(y.shape)
if y.isnull().any():
    y = y.dropna()
    x = x.loc[y.index]
print(x.shape)
print(y.shape)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
def evaluate_model(x_train, x_test, y_train, y_test):
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return mean_absolute_error(y_test, y_pred)
def forward_feature_selection(x_train, x_test, y_train, y_test):
    selected_features = []
    remaining_features = list(x_train.columns)
    best_mae = float('inf')
    mae_progression = []
    while remaining_features:
        mae_candidates = {}
        for feature in remaining_features:
            current_features = selected_features + [feature]
            mae = evaluate_model(x_train[current_features], x_test[current_features], y_train, y_test)
            mae_candidates[feature] = mae
        best_feature = min(mae_candidates, key=mae_candidates.get)
        best_feature_mae = mae_candidates[best_feature]
        if best_feature_mae < best_mae:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_mae = best_feature_mae
            mae_progression.append(best_mae)
            print(f"Selected feature: {best_feature}, MAE: {best_feature_mae}")
        else:
            break
    return selected_features, mae_progression

x_dropped = x.dropna()
x_train, x_test, y_train, y_test = train_test_split(x_dropped, y[x_dropped.index], test_size=0.3, random_state=42)
selected_features, mae_progression = forward_feature_selection(x_train, x_test, y_train, y_test)
print(f"Optimal set of features: {selected_features}")
print(f"MAE progression: {mae_progression}")
final_mae = evaluate_model(x_train, x_test, y_train, y_test)
print(f" MAE with all features: {final_mae}")
final_mae = evaluate_model(x_train[selected_features], x_test[selected_features], y_train, y_test)
print(f"Final MAE with selected features: {final_mae}")


(34857, 19)
(34857,)
(34854, 19)
(34854,)
Selected feature: Regionname, MAE: 3419.9371552030493
Selected feature: Type, MAE: 3388.3608408268474
Selected feature: Postcode, MAE: 3386.325435298081
Selected feature: Bathroom, MAE: 3383.149568388873
Selected feature: Distance, MAE: 3381.806063508436
Selected feature: Date, MAE: 3381.559820194518
Optimal set of features: ['Regionname', 'Type', 'Postcode', 'Bathroom', 'Distance', 'Date']
MAE progression: [3419.9371552030493, 3388.3608408268474, 3386.325435298081, 3383.149568388873, 3381.806063508436, 3381.559820194518]
 MAE with all features: 3462.358899620885
Final MAE with selected features: 3381.559820194518
